In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import timedelta

import numpy as np
import pandas as pd

import pyomo.environ as opt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from mpc import build_linear_optimizer, build_non_linear_optimizer, simses_factory, load_price_timeseries

In [ ]:
pd.options.plotting.backend = "plotly"
template = "plotly_dark"

In [ ]:
def recover_ecm(model):
    return pd.DataFrame(
        data={
            "power": np.array([opt.value(model.bess.power[t]) for t in model.time]),
            "power_dc": np.array([opt.value(model.bess.power_dc[t]) for t in model.time]),
            "soc": np.array([opt.value(model.bess.soc[t]) for t in model.time]),
            "i": np.array([opt.value(model.bess.i[t]) for t in model.time]),
            "v": np.array([opt.value(model.bess.v[t]) for t in model.time]),
            "ocv": np.array([opt.value(model.bess.ocv[t]) for t in model.time]),
        }
    )

In [ ]:
def simulate_results(res, dt=900):
    sim = simses_factory(start_soc=0.0)
    sim_steps = int(dt/60)
    r = sim.storage.state.rint

    df = pd.DataFrame()

    for t in res.index:
        power_opt = np.round(res.loc[t, "power"])
        # power_dc_opt = res.loc[t, "power_dc"]
        # i_opt = res.loc[t, "i"]
        soc_opt = res.loc[t, "soc"]

        # converter_losses_opt = power_opt - power_dc_opt
        # battery_losses_opt = r * i_opt ** 2

        for step in range(sim_steps):
            time = t + (step * timedelta(seconds=60))
            sim.update(power_setpoint=power_opt, dt=60)

            soc_sim = sim.storage.state.soc
            power_sim = sim.state.power
            converter_losses = sim.state.loss
            battery_losses = sim.storage.state.loss

            data = {
                "soc_opt": soc_opt,
                "soc_sim": soc_sim,
                "power_opt": power_opt,
                "power_sim": power_sim,
                "converter_losses": converter_losses,
                "battery_losses": battery_losses,
                # "converter_losses_opt": converter_losses_opt,
                # "battery_losses_opt": battery_losses_opt,
            }
            df = pd.concat([df, pd.DataFrame(index=[time], data=[data])])


    return df

In [ ]:
def benchmark(price, dt, horizon=24):
    t0 = price.index[0]
    price = price.loc[t0:(t0 + timedelta(hours=horizon))]
    price = price.resample(timedelta(seconds=dt)).ffill()

    # opt_nl = build_non_linear_optimizer(price, max_fec=2*(horizon/24))
    opt_nl = build_linear_optimizer(price, max_fec=2*(horizon/24))
    opt_nl.solve({"bess":{"soc_start": 0.0}})
    # res_opt = recover_ecm(opt_nl.model)
    res_opt = opt_nl.recover_results()
    res_opt.index = price.index

    df = simulate_results(res_opt, dt=dt)
    return df

In [ ]:
price = load_price_timeseries("../data/intraday_prices/electricity_prices_germany_2021.csv")

In [ ]:
res = {}

In [ ]:
res[15] = benchmark(price, dt=900, horizon=24) 

In [ ]:
res[5] = benchmark(price, dt=300, horizon=24)

In [ ]:
res[3] = benchmark(price, dt=180, horizon=24)

In [ ]:
# for dt in (15, 5, 3):
#     print(dt)
#     print(res[dt][["battery_losses", "battery_losses_opt"]].sum())
#     print()

In [ ]:
for dt in (15, 5, 3):
    # fig = res[dt][["battery_losses", "battery_losses_opt"]].plot(template=template)
    # fig = res[dt][["soc_opt", "soc_sim"]].plot(template=template)
    fig = res[dt][["power_opt", "power_sim"]].plot(template=template)
    fig.update_layout(title=f"{dt} min")
    fig.show()

In [ ]:
def plot_comparison(res, key, **kwargs):
    fig = make_subplots(cols=1, rows=1)
    fig.update_layout(**kwargs)
    
    for (dt, df) in res.items():
        fig.add_trace(go.Scatter(x=df.index, y=df[key], name=dt))

    return fig

In [ ]:
plot_comparison(res,"power_opt", template=template)